In [4]:
# Question 9: Distance Matrix Calculation

In [16]:
import pandas as pd
import numpy as np

def calculate_distance_matrix(file_path):
    # Step 1: Load the dataset
    df = pd.read_csv(r'C:\Users\ACER\Downloads\dataset-2.csv')
    
    # Assuming the dataset has columns: 'id_start', 'id_end', and 'distance'
    
    # Step 2: Create a unique list of IDs
    unique_ids = pd.unique(df[['id_start', 'id_end']].values.ravel('K'))
    
    # Step 3: Initialize a distance matrix with infinity values (for unknown distances)
    distance_matrix = pd.DataFrame(np.inf, index=unique_ids, columns=unique_ids)
    
    # Step 4: Fill the distance matrix with known distances
    for _, row in df.iterrows():
        start_id = row['id_start']
        end_id = row['id_end']
        distance = row['distance']
        
        # Set the distance for start to end
        distance_matrix.at[start_id, end_id] = distance
        
        # Set the distance for end to start (to make it symmetric)
        distance_matrix.at[end_id, start_id] = distance
    
    # Step 5: Apply Floyd-Warshall algorithm for all-pairs shortest paths
    for k in unique_ids:
        for i in unique_ids:
            for j in unique_ids:
                # Check if passing through 'k' offers a shorter path from 'i' to 'j'
                new_distance = distance_matrix.at[i, k] + distance_matrix.at[k, j]
                if new_distance < distance_matrix.at[i, j]:
                    distance_matrix.at[i, j] = new_distance
                    distance_matrix.at[j, i] = new_distance  # Ensure symmetry

    # Step 6: Set diagonal values to zero (distance from a location to itself is zero)
    np.fill_diagonal(distance_matrix.values, 0)

    return distance_matrix

# Example usage
file_path = r'C:\Users\ACER\Downloads\dataset-2.csv'
distance_matrix = calculate_distance_matrix(file_path)

# Display the resulting distance matrix
print("Distance Matrix:")
print(distance_matrix)

    

Distance Matrix:
         1001400  1001402  1001404  1001406  1001408  1001410  1001412  \
1001400      0.0      9.7     29.9     45.9     67.6     78.7     94.3   
1001402      9.7      0.0     20.2     36.2     57.9     69.0     84.6   
1001404     29.9     20.2      0.0     16.0     37.7     48.8     64.4   
1001406     45.9     36.2     16.0      0.0     21.7     32.8     48.4   
1001408     67.6     57.9     37.7     21.7      0.0     11.1     26.7   
1001410     78.7     69.0     48.8     32.8     11.1      0.0     15.6   
1001412     94.3     84.6     64.4     48.4     26.7     15.6      0.0   
1001414    112.5    102.8     82.6     66.6     44.9     33.8     18.2   
1001416    125.7    116.0     95.8     79.8     58.1     47.0     31.4   
1001418    139.3    129.6    109.4     93.4     71.7     60.6     45.0   
1001420    152.2    142.5    122.3    106.3     84.6     73.5     57.9   
1001422    161.8    152.1    131.9    115.9     94.2     83.1     67.5   
1001424    173.2    1

In [ ]:
# Question 10: Unroll Distance Matrix

In [17]:
import pandas as pd

def unroll_distance_matrix(distance_matrix):
    # Step 1: Get the unique IDs (index and columns are the same in the distance matrix)
    ids = distance_matrix.index
    
    # Step 2: Create a list to store the unrolled data
    unrolled_data = []
    
    # Step 3: Iterate over all pairs of IDs and collect the distances
    for id_start in ids:
        for id_end in ids:
            # Skip the diagonal (same start and end ID)
            if id_start != id_end:
                distance = distance_matrix.at[id_start, id_end]
                # Append the id_start, id_end, and the corresponding distance
                unrolled_data.append([id_start, id_end, distance])
    
    # Step 4: Convert the unrolled data into a DataFrame
    unrolled_df = pd.DataFrame(unrolled_data, columns=['id_start', 'id_end', 'distance'])
    
    return unrolled_df

# Example usage with the distance_matrix created from the previous function
unrolled_df = unroll_distance_matrix(distance_matrix)

# Display the resulting DataFrame
print(unrolled_df)


      id_start   id_end  distance
0      1001400  1001402       9.7
1      1001400  1001404      29.9
2      1001400  1001406      45.9
3      1001400  1001408      67.6
4      1001400  1001410      78.7
...        ...      ...       ...
1801   1001472  1001464      45.8
1802   1001472  1001466      37.3
1803   1001472  1001468      26.6
1804   1001472  1001470      16.0
1805   1001472  1001437     202.2

[1806 rows x 3 columns]


In [ ]:
# Question 11: Finding IDs within Percentage Threshold

In [19]:
import pandas as pd

def find_ids_within_ten_percentage_threshold(unrolled_df, reference_id):
    # Step 1: Filter the rows where id_start equals the reference value
    reference_rows = unrolled_df[unrolled_df['id_start'] == reference_id]
    
    # Step 2: Calculate the average distance for the reference value
    reference_avg_distance = reference_rows['distance'].mean()
    
    # Step 3: Calculate the 10% threshold (lower and upper bounds)
    lower_threshold = reference_avg_distance * 0.9
    upper_threshold = reference_avg_distance * 1.1
    
    # Step 4: Group by id_start and calculate the average distance for each group
    avg_distances_by_id = unrolled_df.groupby('id_start')['distance'].mean().reset_index()
    
    # Step 5: Filter IDs whose average distance lies within the 10% threshold
    ids_within_threshold = avg_distances_by_id[
        (avg_distances_by_id['distance'] >= lower_threshold) &
        (avg_distances_by_id['distance'] <= upper_threshold)
    ]['id_start'].tolist()
    
    # Step 6: Sort the resulting IDs and return
    return sorted(ids_within_threshold)

# Example usage with the unrolled_df created from the previous function
reference_id = 'A'  # Replace with actual reference id
result_ids = find_ids_within_ten_percentage_threshold(unrolled_df, reference_id)

# Display the sorted list of IDs within the 10% threshold
print(result_ids)


[]


In [ ]:
# Question 12: Calculate Toll Rate


In [22]:
import pandas as pd

def calculate_toll_rate(unrolled_df):
    # Step 1: Define the rate coefficients for each vehicle type
    rate_coefficients = {
        'moto': 0.8,
        'car': 1.2,
        'rv': 1.5,
        'bus': 2.2,
        'truck': 3.6
    }
    
    # Step 2: Add the new columns to the DataFrame by multiplying the distance by the respective rate
    for vehicle_type, rate in rate_coefficients.items():
        unrolled_df[vehicle_type] = unrolled_df['distance'] * rate
    
    return unrolled_df

# Example usage with the unrolled_df created from the previous function
toll_rates_df = calculate_toll_rate(unrolled_df)

# Display the resulting DataFrame with toll rates for each vehicle type
print(toll_rates_df)


      id_start   id_end  distance    moto     car      rv     bus   truck
0      1001400  1001402       9.7    7.76   11.64   14.55   21.34   34.92
1      1001400  1001404      29.9   23.92   35.88   44.85   65.78  107.64
2      1001400  1001406      45.9   36.72   55.08   68.85  100.98  165.24
3      1001400  1001408      67.6   54.08   81.12  101.40  148.72  243.36
4      1001400  1001410      78.7   62.96   94.44  118.05  173.14  283.32
...        ...      ...       ...     ...     ...     ...     ...     ...
1801   1001472  1001464      45.8   36.64   54.96   68.70  100.76  164.88
1802   1001472  1001466      37.3   29.84   44.76   55.95   82.06  134.28
1803   1001472  1001468      26.6   21.28   31.92   39.90   58.52   95.76
1804   1001472  1001470      16.0   12.80   19.20   24.00   35.20   57.60
1805   1001472  1001437     202.2  161.76  242.64  303.30  444.84  727.92

[1806 rows x 8 columns]


In [ ]:
# Question 13: Calculate Time-Based Toll Rates

In [23]:
import pandas as pd
import datetime

def calculate_time_based_toll_rates(toll_rates_df):
    # Step 1: Define days and their respective discount factors
    weekdays = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']
    weekends = ['Saturday', 'Sunday']
    
    # Define time ranges for weekdays
    time_ranges = [
        (datetime.time(0, 0), datetime.time(10, 0), 0.8),     # 00:00 - 10:00
        (datetime.time(10, 0), datetime.time(18, 0), 1.2),    # 10:00 - 18:00
        (datetime.time(18, 0), datetime.time(23, 59, 59), 0.8) # 18:00 - 23:59
    ]
    
    # Step 2: Prepare the new DataFrame to store results
    new_rows = []

    # Step 3: Iterate through each unique (id_start, id_end) pair and time ranges
    for _, row in toll_rates_df.iterrows():
        id_start = row['id_start']
        id_end = row['id_end']
        distances = {
            'moto': row['moto'],
            'car': row['car'],
            'rv': row['rv'],
            'bus': row['bus'],
            'truck': row['truck']
        }
        
        for day in weekdays:
            for start_time, end_time, discount in time_ranges:
                # Create the full 24-hour period for each day
                new_rows.append({
                    'id_start': id_start,
                    'id_end': id_end,
                    'start_day': day,
                    'start_time': start_time,
                    'end_day': day,
                    'end_time': end_time,
                    **{vehicle: distance * discount for vehicle, distance in distances.items()}
                })

        # Apply the weekend discount for Saturday and Sunday
        for weekend_day in weekends:
            for start_time in [datetime.time(0, 0)]:  # Only need one start time
                end_time = datetime.time(23, 59, 59)  # All day
                discount = 0.7
                new_rows.append({
                    'id_start': id_start,
                    'id_end': id_end,
                    'start_day': weekend_day,
                    'start_time': start_time,
                    'end_day': weekend_day,
                    'end_time': end_time,
                    **{vehicle: distance * discount for vehicle, distance in distances.items()}
                })
    
    # Step 4: Create a new DataFrame from the new rows
    time_based_toll_rates_df = pd.DataFrame(new_rows)

    return time_based_toll_rates_df

# Example usage with the toll_rates_df DataFrame created from the previous function
time_based_toll_rates_df = calculate_time_based_toll_rates(toll_rates_df)

# Display the resulting DataFrame with time-based toll rates
print(time_based_toll_rates_df)


        id_start     id_end start_day start_time   end_day  end_time     moto  \
0      1001400.0  1001402.0    Monday   00:00:00    Monday  10:00:00    6.208   
1      1001400.0  1001402.0    Monday   10:00:00    Monday  18:00:00    9.312   
2      1001400.0  1001402.0    Monday   18:00:00    Monday  23:59:59    6.208   
3      1001400.0  1001402.0   Tuesday   00:00:00   Tuesday  10:00:00    6.208   
4      1001400.0  1001402.0   Tuesday   10:00:00   Tuesday  18:00:00    9.312   
...          ...        ...       ...        ...       ...       ...      ...   
30697  1001472.0  1001437.0    Friday   00:00:00    Friday  10:00:00  129.408   
30698  1001472.0  1001437.0    Friday   10:00:00    Friday  18:00:00  194.112   
30699  1001472.0  1001437.0    Friday   18:00:00    Friday  23:59:59  129.408   
30700  1001472.0  1001437.0  Saturday   00:00:00  Saturday  23:59:59  113.232   
30701  1001472.0  1001437.0    Sunday   00:00:00    Sunday  23:59:59  113.232   

           car      rv     